Setup the openai api key in env

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

To use OpenAI language models

In [2]:
import dspy
gpt3_turbo = dspy.OpenAI(model='gpt-3.5-turbo-1106', max_tokens=300)
dspy.configure(lm=gpt3_turbo)

/home/cch1006/langchain-startup/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


You can simply call the LM with a string to give it a raw prompt.

In [3]:
# This is almost never the recommended way to interact with LMs in DSPy, but it is allowed.
gpt3_turbo("hello! this is a raw prompt to GPT-3.5")

['Hello! How can I assist you today?']

Using the LM with DSPy signatures.

In [5]:
# Define a module (ChainOfThought) and assign it a signature (return an answer, given a question).
qa = dspy.ChainOfThought('question -> answer')

# Run with the default LM configured with `dspy.configure` above.
response = qa(question="How many floors are in the castle David Gregory inherited?")
print(response.answer)

The castle David Gregory inherited has 5 floors.


Using multiple LMs at once.

In [6]:
# Run with the default LM configured above, i.e. GPT-3.5
response = qa(question="How many floors are in the castle David Gregory inherited?")
print('GPT-3.5:', response.answer)

gpt4_turbo = dspy.OpenAI(model='gpt-4-1106-preview', max_tokens=300)

# Run with GPT-4 instead
with dspy.context(lm=gpt4_turbo):
    response = qa(question="How many floors are in the castle David Gregory inherited?")
    print('GPT-4-turbo:', response.answer)

GPT-3.5: The castle David Gregory inherited has 5 floors.
GPT-4-turbo: The number of floors in the castle David Gregory inherited cannot be determined with the information provided.


Tips and Tricks.

In DSPy, all LM calls are cached. If you repeat the same call, you will get the same outputs.

To generate 5 outputs, you can use n=5 in the module constructor, or pass config=dict(n=5) when invoking the module.

In [8]:
qa = dspy.ChainOfThought('question -> answer', n=5)

response = qa(question="How many floors are in the castle David Gregory inherited?")
response.completions.answer

['The castle David Gregory inherited has a total of 5 floors.',
 'The castle David Gregory inherited has 14 floors.',
 'The castle David Gregory inherited has 4 floors.',
 'The castle David Gregory inherited has 4 floors.',
 'The number of floors in the castle David Gregory inherited is 4.']

In [9]:
for idx in range(5):
    response = qa(question="How many floors are in the castle David Gregory inherited?", config=dict(temperature=0.7+0.0001*idx))
    print(f'{idx+1}.', response.answer)

1. The castle David Gregory inherited has a total of 5 floors.
2. The castle David Gregory inherited has 4 floors.
3. The castle David Gregory inherited has four floors.
4. The castle David Gregory inherited has 5 floors.
5. The castle David Gregory inherited has 5 floors.
